# Import


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
!pip install plotly_express

In [25]:
!pip install geopandas

In [26]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'

trips_2019_10_10.csv  trips_2019_2_10.csv  trips_2019_6_12.csv
trips_2019_10_11.csv  trips_2019_2_11.csv  trips_2019_6_13.csv
trips_2019_10_12.csv  trips_2019_2_12.csv  trips_2019_6_14.csv
trips_2019_10_13.csv  trips_2019_2_13.csv  trips_2019_6_15.csv
trips_2019_10_14.csv  trips_2019_2_14.csv  trips_2019_6_16.csv
trips_2019_10_15.csv  trips_2019_2_15.csv  trips_2019_6_17.csv
trips_2019_10_16.csv  trips_2019_2_16.csv  trips_2019_6_18.csv
trips_2019_10_17.csv  trips_2019_2_17.csv  trips_2019_6_19.csv
trips_2019_10_18.csv  trips_2019_2_18.csv  trips_2019_6_1.csv
trips_2019_10_19.csv  trips_2019_2_19.csv  trips_2019_6_20.csv
trips_2019_10_1.csv   trips_2019_2_1.csv   trips_2019_6_21.csv
trips_2019_10_20.csv  trips_2019_2_20.csv  trips_2019_6_22.csv
trips_2019_10_21.csv  trips_2019_2_21.csv  trips_2019_6_23.csv
trips_2019_10_22.csv  trips_2019_2_22.csv  trips_2019_6_24.csv
trips_2019_10_23.csv  trips_2019_2_23.csv  trips_2019_6_25.csv
trips_2019_10_24.csv  trips_2019_2_24.csv  trips_2019_6_

In [27]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'
export_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

# Map trip to way sequence with ckd-tree

load trips data

In [28]:
trips_df = pd.read_csv(trips_data_path+'/trips_2019_1_1.csv', parse_dates=['time_stamp'])
trips_df[:5]

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0


load way segment

In [29]:
way_segment_drop_dup_df = pd.read_csv(road_data_path + 'way_and_road_segment_lazy_drop.csv')
way_segment_drop_dup_df[:5]

,mid_lat,mid_lon,way_id
0,17.436395,102.802536,23730732
1,16.471186,103.040827,23747253
2,14.520287,104.504161,23747922
3,14.509161,104.529506,23747922
4,14.504591,104.575137,23747922


convert from df to gdf

In [30]:
import itertools
from operator import itemgetter

import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString

# from pd to gpd
trips_gdf = gpd.GeoDataFrame(
    trips_df, geometry=gpd.points_from_xy(trips_df.lat, trips_df.lon))

# create linestring of way_id
way_segment_drop_dup_gdf = gpd.GeoDataFrame(
  way_segment_drop_dup_df, geometry=gpd.points_from_xy(
      way_segment_drop_dup_df.mid_lat,
      way_segment_drop_dup_df.mid_lon
    )
  )

way_id_list = []
for way_id in pd.unique(way_segment_drop_dup_gdf.way_id):
  line = way_segment_drop_dup_gdf[way_segment_drop_dup_gdf.way_id == way_id]['geometry'].to_list()
  if len(line) < 2:
    line = line*2
  way_id_list.append([way_id, LineString(line)])
way_id_list

way_segment_drop_dup_gdf = gpd.GeoDataFrame(way_id_list, columns=['way_id', 'geometry'])

In [31]:
trips_gdf[:5]

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,geometry
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0,POINT (16.482 104.050)
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0,POINT (16.491 104.052)
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0,POINT (16.503 104.055)
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0,POINT (16.514 104.056)
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0,POINT (16.525 104.056)


In [32]:
way_segment_drop_dup_gdf[:5]

,way_id,geometry
0,23730732,"LINESTRING (17.436 102.803, 17.436 102.803)"
1,23747253,"LINESTRING (16.471 103.041, 16.471 103.041)"
2,23747922,"LINESTRING (14.520 104.504, 14.509 104.530, 14..."
3,23749257,"LINESTRING (17.172 104.099, 17.187 104.095)"
4,23749339,"LINESTRING (16.611 103.620, 16.634 103.628)"


map nearest way_id

In [33]:
def ckdnearest(gdfA, gdfB, gdfB_cols=['way_id']):
    A = np.concatenate(
        [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    B = [np.array(geom.coords) for geom in gdfB.geometry.to_list()]
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix)
    gdf = pd.concat(
        [gdfA, gdfB.loc[idx, gdfB_cols].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

trips_gdf_nearest_way = ckdnearest(trips_gdf, way_segment_drop_dup_gdf)
trips_gdf_nearest_way

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,geometry,way_id,dist
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0,POINT (16.482 104.050),340294191,0.004308
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0,POINT (16.491 104.052),92193150,0.000498
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0,POINT (16.503 104.055),402348153,0.004740
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0,POINT (16.514 104.056),92193150,0.008590
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0,POINT (16.525 104.056),92193150,0.002386
...,...,...,...,...,...,...,...,...,...,...
195857,2019-01-01 21:51:58,003000300000359769033400604,14.835272,101.550918,52,7.0,2098,POINT (14.835 101.551),82374801,0.001614
195858,2019-01-01 21:53:56,003000300000359769033400604,14.846673,101.562518,61,7.0,2098,POINT (14.847 101.563),82374801,0.001829
195859,2019-01-01 21:56:00,003000300000359769033400604,14.850647,101.572147,59,7.0,2098,POINT (14.851 101.572),82374801,0.007574
195860,2019-01-01 21:57:55,003000300000359769033400604,14.844158,101.590907,60,7.0,2098,POINT (14.844 101.591),82374801,0.005860


In [34]:
# export_df = trips_gdf_nearest_way.drop(columns=['geometry'])
# export_df.to_csv(export_data_path+'/trips_2019_1_1.csv', index=False)